In [1]:
# imports

# !pip install tf-models-official
# import tqdm
import tensorflow as tf
# import tensorflow_datasets as tfds
# import tensorflow_models as tfm
import numpy as np
import json
import os
import cv2
import re
from operator import itemgetter

2024-10-05 22:38:45.923253: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-05 22:38:46.065419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1728160726.128476    5166 cuda_dnn.cc:8322] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1728160726.148312    5166 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-05 22:38:46.299512: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# download the dataset

# !wget http://images.cocodataset.org/zips/train2014.zip
# !wget http://images.cocodataset.org/zips/val2014.zip
# !wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
# !wget http://images.cocodataset.org/zips/test2014.zip
# !wget http://images.cocodataset.org/annotations/image_info_test2014.zip


In [ ]:
# unzip the dataset files

# !pip install unzip
# !unzip /home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/val2014.zip


In [2]:
# dataset paths

train_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/train2014/"
val_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/val2014/"
test_imgs_path = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/test2014/"
annotations = "/home/samer/Desktop/Beedoo/FCOS/FCOS_Implementation/COCO2014/annotations/"

In [85]:
images_dict = dict()
image_list = []
counter = 0
# load the images
for file in sorted(os.listdir(train_imgs_path)):
    if counter < 1000:
        image_path = os.path.join(train_imgs_path, file)
        image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (640, 640))
        image = np.array(image)
        image = image.astype('float32')
        image /= 255
        if len(image.shape) < 3:
            continue
        # plt.imshow(image)
        # plt.show()
        file_key = str(file).split('.')[0]
        imageid = re.findall("[0-9]+", file_key)
        imageid = str(imageid[1]).lstrip("_0")
        print(imageid)
        images_dict[f'{imageid}'] = image           # create a dictionary of the image titles and themselves
        image_list.append(image)
        print(file_key)
        counter += 1
    else:
        break

9
COCO_train2014_000000000009
25
COCO_train2014_000000000025
30
COCO_train2014_000000000030
34
COCO_train2014_000000000034
36
COCO_train2014_000000000036
49
COCO_train2014_000000000049
61
COCO_train2014_000000000061
64
COCO_train2014_000000000064
71
COCO_train2014_000000000071
72
COCO_train2014_000000000072
77
COCO_train2014_000000000077
78
COCO_train2014_000000000078
81
COCO_train2014_000000000081
89
COCO_train2014_000000000089
92
COCO_train2014_000000000092
94
COCO_train2014_000000000094
109
COCO_train2014_000000000109
110
COCO_train2014_000000000110
113
COCO_train2014_000000000113
127
COCO_train2014_000000000127
138
COCO_train2014_000000000138
142
COCO_train2014_000000000142
144
COCO_train2014_000000000144
149
COCO_train2014_000000000149
151
COCO_train2014_000000000151
154
COCO_train2014_000000000154
165
COCO_train2014_000000000165
194
COCO_train2014_000000000194
201
COCO_train2014_000000000201
247
COCO_train2014_000000000247
250
COCO_train2014_000000000250
260
COCO_train2014_000000

KeyboardInterrupt: 

In [31]:
def concat(list1, list2):
    print("this is list1", list1, "list1 type", type(list1), "this is list2", list2, "list2 type", type(list2))
    if type(list1[0]) == list and type(list2) == list:
        # print("this is list1", list1[0])
        alist = list1
        alist.append(list2)
        # print("this is first case")
    # elif type(list1[0]) != list and type(list2) == list:
    #     # print("this is list1",list1[0])
    #     alist = [x for x in [list1, list2]]
    elif type(list2) == int:
        alist = [list1]
        alist.append(list2)
        # alist = [alist]
    print("this is alist", alist)
    return alist

concat([1.08, 187.69, 611.59, 285.84],51)#, [[1.08, 187.69, 611.59, 285.84],51]))

this is list1 [1.08, 187.69, 611.59, 285.84] list1 type <class 'list'> this is list2 51 list2 type <class 'int'>
this is alist [[1.08, 187.69, 611.59, 285.84], 51]


[[1.08, 187.69, 611.59, 285.84], 51]

In [32]:
labels_list = []
bounding_boxes = []
counter = 0
dont_care = [0,0,0,0,0,0,0,0,0]
annotation_path = os.path.join(annotations, "instances_train2014.json")
boxes = dict()
counter = 0

with open(annotation_path, "r") as f:
    json_obj = json.load(f)
    sorted_imageids = sorted(json_obj["annotations"], key=itemgetter("image_id"))
# draw the bounding boxes
    for line in sorted_imageids:      # iterate over the lines of the label file, where each line is a box with it's label
        if counter < 1000:
            # print(line)
            if not boxes.get(line["image_id"]):
                # print(line['bbox'], "  ", line["category_id"])
                # print(concat(line["bbox"], line["category_id"]))
                boxes[line["image_id"]] = concat(line["bbox"], line["category_id"])
            elif boxes.get(line["image_id"]):
                boxes[line["image_id"]] = concat(boxes[line["image_id"]], concat(line["bbox"], line["category_id"]))
            counter += 1

    print(boxes)



this is list1 [1.08, 187.69, 611.59, 285.84] list1 type <class 'list'> this is list2 51 list2 type <class 'int'>
this is alist [[1.08, 187.69, 611.59, 285.84], 51]
this is list1 [311.73, 4.31, 319.28, 228.68] list1 type <class 'list'> this is list2 51 list2 type <class 'int'>
this is alist [[311.73, 4.31, 319.28, 228.68], 51]
this is list1 [[1.08, 187.69, 611.59, 285.84], 51] list1 type <class 'list'> this is list2 [[311.73, 4.31, 319.28, 228.68], 51] list2 type <class 'list'>
this is alist [[1.08, 187.69, 611.59, 285.84], 51, [[311.73, 4.31, 319.28, 228.68], 51]]
this is list1 [249.6, 229.27, 316.24, 245.08] list1 type <class 'list'> this is list2 56 list2 type <class 'int'>
this is alist [[249.6, 229.27, 316.24, 245.08], 56]
this is list1 [[1.08, 187.69, 611.59, 285.84], 51, [[311.73, 4.31, 319.28, 228.68], 51]] list1 type <class 'list'> this is list2 [[249.6, 229.27, 316.24, 245.08], 56] list2 type <class 'list'>
this is alist [[1.08, 187.69, 611.59, 285.84], 51, [[311.73, 4.31, 319

In [3]:
# loading and procseing the dataset
# json keys = info, images, licenses, annotations, categories
annotation_path = os.path.join(annotations, "instances_val2014.json")
with open(annotation_path, "r") as f:
    json_obj = json.load(f)
    print(json_obj["annotations"][0])
    
    
# dataset = tfds.load("coco")

{'segmentation': [[239.97, 260.24, 222.04, 270.49, 199.84, 253.41, 213.5, 227.79, 259.62, 200.46, 274.13, 202.17, 277.55, 210.71, 249.37, 253.41, 237.41, 264.51, 242.54, 261.95, 228.87, 271.34]], 'area': 2765.1486500000005, 'iscrowd': 0, 'image_id': 558840, 'bbox': [199.84, 200.46, 77.71, 70.88], 'category_id': 58, 'id': 156}


In [ ]:
# model backbone

tfm.vision.backbones.ResNet(
    model_id: int,
    input_specs: tf.keras.layers.InputSpec = layers.InputSpec(shape=[None, None, None, 3]),
    depth_multiplier: float = 1.0,
    stem_type: str = &#x27;v0',
    resnetd_shortcut: bool = False,
    replace_stem_max_pool: bool = False,
    se_ratio: Optional[float] = None,
    init_stochastic_depth_rate: float = 0.0,
    scale_stem: bool = True,
    activation: str = &#x27;relu',
    use_sync_bn: bool = False,
    norm_momentum: float = 0.99,
    norm_epsilon: float = 0.001,
    kernel_initializer: str = &#x27;VarianceScaling',
    kernel_regularizer: Optional[tf.keras.regularizers.Regularizer] = None,
    bias_regularizer: Optional[tf.keras.regularizers.Regularizer] = None,
    bn_trainable: bool = True)

In [2]:
tfm.vision.decoders.FPN(
    input_specs: Mapping[str, tf.TensorShape],
    min_level: int = 3,
    max_level: int = 7,
    num_filters: int = 256,
    fusion_type: str = 'sum',
    use_separable_conv: bool = False,
    use_keras_layer: bool = False,
    activation: str = 'relu',
    use_sync_bn: bool = False,
    norm_momentum: float = 0.99,
    norm_epsilon: float = 0.001,
    kernel_initializer: str = 'VarianceScaling',
    kernel_regularizer: Optional[tf.keras.regularizers.Regularizer] = None,
    bias_regularizer: Optional[tf.keras.regularizers.Regularizer] = None,
    **kwargs
)

SyntaxError: invalid syntax (2388210152.py, line 2)